# DS 710 Final Project - Patrick Christ
## Final Fantasy XIV Sentiment Analysis
Name: Patrick Christ

Date: 12/16/2020

*Note: For consistency, I ended up using the REST API to find both Tweets before the patch and after. This is different from my project proposal but I believe still achieves the desired result.*

### Data Collection Setup

In [2]:
#import relevant libraries
import tweepy
import csv
import pandas as pd
import numpy as np
from textblob import TextBlob

In [2]:
#grab my Twitter credentials
%run C:\Users\patse\tweeter.py

In [3]:
#Authenticate and connect to the Twitter API
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)
api = tweepy.API(auth)

In [8]:
#This function will gather the tweets I need. I used the code provided from the example with some
#tweaks for FFXIV related data
def get_tweets(num_needed,until):
    
    tweet_list = []
    last_id = -1 # id of last tweet seen
    while len(tweet_list) < num_needed:
        try:
            new_tweets = api.search(q = '#%23ffxiv',until=until,lang="en",count = 100,max_id = str(last_id - 1))
        except tweepy.TweepError as e:
            print("Error", e)
            break
        else:
            if not new_tweets:
                print("Could not find any more tweets!")
                break
            tweet_list.extend(new_tweets)
            last_id = new_tweets[-1].id

    return tweet_list

In [81]:
#Here I break each tweet into the columns I want in my dataframe

##Index: Column Name
#0 Name
#1 Month
#2 Day
#3 Hour
#4 Minute
#5 Second
#6 URL
#7 Media
#8 Favorite_Count
#9 Retweet_Count
#10 Text
#11 Language (filtered to find EN tweets)
#12 Retweeted?

def tweets_to_df(tweet_list):
    #preallocate my variables and define what column names I'm interested in
    tw_data = []
    One_Tweet = []
    column_names = ["Name","Month","Day","Hour","Minute","Second","Url",
                    "Media","Favorite_Count","Retweet_Count","Text","Language","Retweeted?"]
    #loop through each tweet and create a list by tweet of relevant info
    for tweet in tweet_list:
        One_Tweet = [tweet.author.name,
                    tweet.created_at.month,
                    tweet.created_at.day,
                    tweet.created_at.hour,
                    tweet.created_at.minute,
                    tweet.created_at.second,
                    tweet.entities.get("urls")[0].get("url") if tweet.entities.get("urls") else np.nan,
                    tweet.entities.get("media")[0].get("type") if tweet.entities.get("media") else np.nan,
                    tweet.favorite_count,
                    tweet.retweet_count,
                    tweet.text,
                    tweet.lang,
                    tweet.retweeted]
        tw_data.append(One_Tweet) #append that tweet to my total data

    tweet_df = pd.DataFrame(tw_data,columns=column_names) #turn that data into a data frame and return
    return tweet_df

### Pulling Data from before the Patch on 12/08

In [82]:
#I use each block of code to pull 2000 tweets from each day
#This is the 5th
tweets_06 = get_tweets(2000,until="2020-12-06")
tweet_df_06 = tweets_to_df(tweets_06)
tweet_df_06

Could not find any more tweets!


,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?


In [ ]:
#Pulling Tweets from the 3rd
tweets_04 = get_tweets(2000,until="2020-12-04")
tweet_df_04 = tweets_to_df(tweets_04)
tweet_df_04

In [ ]:
#Pulling Tweets from the 4th
tweets_05 = get_tweets(2000,until="2020-12-05")
tweet_df_05 = tweets_to_df(tweets_05)
tweet_df_05

In [ ]:
#Merge all of my tweets by day into a single data frame
merge = [tweet_df_04,tweet_df_05,tweet_df_06]
PrePatchTweets_df = pd.concat(merge)
PrePatchTweets_df

In [3]:
#I did different parts of this analysis at different times, so I used the following code 
#to "pull" the data frame back in rather than keeping python running for weeks.
####
#PrePatchTweets_df.to_csv('PrePatchTweets.csv',na_rep="",index=False)
####
PrePatchTweets_df = pd.read_csv("PrePatchTweets.csv")
####
DowntimeTweets_df = pd.read_csv("DowntimeTweets.csv")
####
PostPatchTweets_df = pd.read_csv("PostPatchTweets.csv")

In [71]:
#Look at our data to make sure it's what we expect
PrePatchTweets_df

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?
0,巨大金魚🐟Pandaemonium🐼,12,3,23,59,54,NaN,NaN,0,319,"RT @FF_XIV_EN: Oh my, what big eyes you have! ...",en,False
1,Zao Lv81 @ 🍭アザz3ルもん🍭,12,3,23,59,53,NaN,NaN,0,178,RT @AestMirra: Pictures taken around Lakeland....,en,False
2,Izik,12,3,23,59,48,NaN,photo,0,154,RT @WhyMaige: Stack on---\n\n........ the Blac...,en,False
3,Toiletta ♿🚽🎀,12,3,23,59,47,NaN,photo,0,225,RT @mellalyss: I drew the most precious catboy...,en,False
4,Alicia Frostfall,12,3,23,59,45,NaN,photo,11,2,Welcome to Amaurot\n\n#FFXIV #FF14 #ffxivsnaps...,en,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,Mal Helasdóttir | 5.4 anticipation vibration,12,5,16,14,17,NaN,NaN,0,24,RT @JesseCox: Well if you're wondering what so...,en,False
6094,Ghost in a Tux,12,5,16,13,32,NaN,photo,0,885,RT @888beru: i wanted to paint estinien.. #ffx...,en,False
6095,SpiritsUnrivaled,12,5,16,13,9,NaN,NaN,0,68,RT @alex_moukala: That time Final Fantasy XIV'...,en,False
6096,Xeborah (Xebi) Bisque,12,5,16,13,9,https://t.co/U3UWtpI8J2,NaN,4,0,Look who won the screenshot contest :D\n\nThan...,en,False


### Pulling Data from after the Patch - live on 12/08

In [67]:
#Pulling Tweets from the 7th
tweets_08 = get_tweets(2000,until="2020-12-08")
tweet_df_08 = tweets_to_df(tweets_08)
tweet_df_08

In [76]:
#technically, this pull would have pulled from the 7th, which was a day the game was down for maintenance.
#So I put these tweets in their own category
DowntimeTweets_df = tweet_df_08

In [10]:
#Pulling Tweets from the 8th
#Since I'm using the REST API, this will pull from the end of the day backwards
#meaning we don't have to worry about exact timing of when the patch went live
##For reference, it went live early in the AM CST on Tuesday 12/8
tweets_09 = get_tweets(2000,until="2020-12-09")
tweet_df_09 = tweets_to_df(tweets_09)
tweet_df_09

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?
0,❄Cap Cringle⸙ ❄aether🎁,12,8,23,59,57,NaN,photo,0,283,RT @DaPandaBanda: The Trust Squad™️ \n#ffxiv #...,en,False
1,JONΛH,12,8,23,59,2,NaN,NaN,11,0,i like fandaniel more than i should 💦 #ffxiv,en,False
2,Stephen Walters,12,8,23,58,5,NaN,NaN,0,781,"RT @FF_XIV_EN: #FFXIV Patch 5.4, Futures Rewri...",en,False
3,Ashley,12,8,23,56,27,https://t.co/spDTjhwp3K,NaN,0,0,5.4 patch day for #FFXIV let's goooooooo\n\n...,en,False
4,APuESyo,12,8,23,56,25,NaN,NaN,0,1102,"RT @winterleigh_art: Dance, dance, until the n...",en,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,Neneko ColorS~🐸💚,12,8,15,18,32,NaN,photo,0,3,RT @EternalDM: An angel has arrived\n\n#ffxiv ...,en,False
2088,💖 Yuwu @ 5.4 Spoilers 💖,12,8,15,18,31,NaN,NaN,0,781,"RT @FF_XIV_EN: #FFXIV Patch 5.4, Futures Rewri...",en,False
2089,🎄 Dia the Rednosed Rain-dia 🎄,12,8,15,18,28,NaN,photo,4,0,"Saggitarius Arrow #ffxiv #PS5Share, #FINALFANT...",en,False
2090,🌿🍁Danica Sweet (ダニカ)🍁🌿,12,8,15,18,28,NaN,photo,4,0,"Man, this right here breaks my gosh-darned hea...",en,False


In [11]:
#Gathering tweets from 12/9
tweets_10 = get_tweets(2000,until="2020-12-10")
tweet_df_10 = tweets_to_df(tweets_10)
tweet_df_10

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?
0,Princess Farron,12,9,23,59,55,https://t.co/3TXo4Xr2hG,NaN,1,0,#FFXIV - #EdenspromiseEternity (1st clear) htt...,en,False
1,connor 🌕 コンナー,12,9,23,59,50,NaN,photo,0,441,RT @ochentiocho: This coat is SO heckin' good....,en,False
2,CJ,12,9,23,59,34,NaN,photo,0,39,RT @organicseren: Really quick paint gtg play ...,en,False
3,FREE BOBBY SHMURDA,12,9,23,59,28,NaN,NaN,0,109,RT @FF_XIV_EN: The team celebrated a great #FF...,en,False
4,witch of the wastes,12,9,23,59,21,NaN,NaN,0,784,RT @FF_XIV_EN: 'Tis the season for a #FFXIV St...,en,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,Rigel,12,9,16,31,46,NaN,NaN,0,784,RT @FF_XIV_EN: 'Tis the season for a #FFXIV St...,en,False
2093,Serge B*L*M/Abolish ICE,12,9,16,31,46,NaN,photo,1,0,Same energy #FFXIV https://t.co/Iw6n3DYvcX,en,False
2094,ai,12,9,16,31,40,NaN,NaN,0,784,RT @FF_XIV_EN: 'Tis the season for a #FFXIV St...,en,False
2095,✨🎄⋆𝓗𝓸𝓵𝓲𝓭𝓪𝔂 Nᴇᴏ⋆🎄✨ 🧡💜,12,9,16,31,19,NaN,photo,0,603,"RT @nonpsart: Sleep, what a wonderful way to p...",en,False


In [60]:
#Gathering tweets from 12/10
tweets_11 = get_tweets(2000,until="2020-12-11")
tweet_df_11 = tweets_to_df(tweets_11)
tweet_df_11

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?
0,prince feli ⚠️ 5.4,12,10,23,59,56,NaN,photo,0,962,RT @zerosshadows: Oracle of Light 🌟\n#FFXIV #F...,en,False
1,Chu @ 5.4 & still simping for sid,12,10,23,59,53,NaN,photo,0,707,RT @zerosshadows: Oracle of Darkness🌑\n#FFXIV ...,en,False
2,✞ｊｏｎａ✞ @ 5.4 BABEY,12,10,23,59,52,NaN,photo,0,707,RT @zerosshadows: Oracle of Darkness🌑\n#FFXIV ...,en,False
3,Emily@xiv hell (5.3 spoilers!),12,10,23,59,51,NaN,photo,0,962,RT @zerosshadows: Oracle of Light 🌟\n#FFXIV #F...,en,False
4,prince feli ⚠️ 5.4,12,10,23,59,50,NaN,photo,0,707,RT @zerosshadows: Oracle of Darkness🌑\n#FFXIV ...,en,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Mateus,12,10,18,2,30,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False
1996,ZoraofWater,12,10,18,2,11,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False
1997,Plama_chan,12,10,18,1,55,NaN,photo,0,0,Do not mind the mannequin behind the case... #...,en,False
1998,[ m i l k y ],12,10,18,1,52,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False


In [61]:
#Merging Post tweets into a single dataframe
merge_post = [tweet_df_09,tweet_df_10,tweet_df_11]
PostPatchTweets_df = pd.concat(merge_post)
PostPatchTweets_df

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?
0,❄Cap Cringle⸙ ❄aether🎁,12,8,23,59,57,NaN,photo,0,283,RT @DaPandaBanda: The Trust Squad™️ \n#ffxiv #...,en,False
1,JONΛH,12,8,23,59,2,NaN,NaN,11,0,i like fandaniel more than i should 💦 #ffxiv,en,False
2,Stephen Walters,12,8,23,58,5,NaN,NaN,0,781,"RT @FF_XIV_EN: #FFXIV Patch 5.4, Futures Rewri...",en,False
3,Ashley,12,8,23,56,27,https://t.co/spDTjhwp3K,NaN,0,0,5.4 patch day for #FFXIV let's goooooooo\n\n...,en,False
4,APuESyo,12,8,23,56,25,NaN,NaN,0,1102,"RT @winterleigh_art: Dance, dance, until the n...",en,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Mateus,12,10,18,2,30,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False
1996,ZoraofWater,12,10,18,2,11,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False
1997,Plama_chan,12,10,18,1,55,NaN,photo,0,0,Do not mind the mannequin behind the case... #...,en,False
1998,[ m i l k y ],12,10,18,1,52,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False


### Adding TextBlob variables

In [91]:
#testing out some textblob functionality to find sentiment
#I had a problem with indices in my merges (multiple items at index [0]). I think I could have found a way around this
#but for my purposes I was able to just remove the index before moving to R
text = str(PostPatchTweets_df["Text"][2092])
test1 = TextBlob(text)
test1

TextBlob("RT @FF_XIV_EN: 'Tis the season for a #FFXIV Starlight Celebration! 🎄

The event will take place from Dec. 14 to Dec. 31! 📅

What starlight…")

In [92]:
test1.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [77]:
#I preallocate a column that I am going to modify using the apply function
PrePatchTweets_df["Polarity"] = 0
PrePatchTweets_df["Subjectivity"] = 0

DowntimeTweets_df["Polarity"] = 0
DowntimeTweets_df["Subjectivity"] = 0

PostPatchTweets_df["Polarity"] = 0
PostPatchTweets_df["Subjectivity"] = 0

#The functions I will use to populate my new columns
def find_polarity(row):
    text = str(row["Text"])
    text = TextBlob(text)
    return text.sentiment.polarity
    
def find_subjectivity(row):
    text = str(row["Text"])
    text = TextBlob(text)
    return text.sentiment.subjectivity

In [78]:
#Run my functions on each data frame
PrePatchTweets_df["Polarity"] = PrePatchTweets_df.apply(find_polarity,axis=1)
PrePatchTweets_df["Subjectivity"] = PrePatchTweets_df.apply(find_subjectivity,axis=1)

DowntimeTweets_df["Polarity"] = DowntimeTweets_df.apply(find_polarity,axis=1)
DowntimeTweets_df["Subjectivity"] = DowntimeTweets_df.apply(find_subjectivity,axis=1)

PostPatchTweets_df["Polarity"] = PostPatchTweets_df.apply(find_polarity,axis=1)
PostPatchTweets_df["Subjectivity"] = PostPatchTweets_df.apply(find_subjectivity,axis=1)

In [4]:
#Break my tweets into different categories
PrePatchTweets_df["Patch_Timing"] = "Before"

DowntimeTweets_df["Patch_Timing"] = "Downtime"

PostPatchTweets_df["Patch_Timing"] = "After"

In [5]:
#merge into a master dataframe
merge_all = [PrePatchTweets_df,DowntimeTweets_df,PostPatchTweets_df]
ffxiv_tweets_df = pd.concat(merge_all)
ffxiv_tweets_df

,Name,Month,Day,Hour,Minute,Second,Url,Media,Favorite_Count,Retweet_Count,Text,Language,Retweeted?,Polarity,Subjectivity,Patch_Timing
0,巨大金魚🐟Pandaemonium🐼,12,3,23,59,54,NaN,NaN,0,319,"RT @FF_XIV_EN: Oh my, what big eyes you have! ...",en,False,0.0,0.10,Before
1,Zao Lv81 @ 🍭アザz3ルもん🍭,12,3,23,59,53,NaN,NaN,0,178,RT @AestMirra: Pictures taken around Lakeland....,en,False,0.0,0.00,Before
2,Izik,12,3,23,59,48,NaN,photo,0,154,RT @WhyMaige: Stack on---\n\n........ the Blac...,en,False,0.0,0.00,Before
3,Toiletta ♿🚽🎀,12,3,23,59,47,NaN,photo,0,225,RT @mellalyss: I drew the most precious catboy...,en,False,0.5,0.75,Before
4,Alicia Frostfall,12,3,23,59,45,NaN,photo,11,2,Welcome to Amaurot\n\n#FFXIV #FF14 #ffxivsnaps...,en,False,0.8,0.90,Before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6184,Mateus,12,10,18,2,30,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False,0.0,0.00,After
6185,ZoraofWater,12,10,18,2,11,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False,0.0,0.00,After
6186,Plama_chan,12,10,18,1,55,NaN,photo,0,0,Do not mind the mannequin behind the case... #...,en,False,-0.4,0.70,After
6187,[ m i l k y ],12,10,18,1,52,NaN,NaN,0,801,"RT @FF_XIV_EN: Hey #FFXIV streamers, this is f...",en,False,0.0,0.00,After


In [7]:
#Export and get ready to look at in R!
PrePatchTweets_df.to_csv('PrePatchTweets.csv',na_rep="",index=False)
DowntimeTweets_df.to_csv('DowntimeTweets.csv',na_rep="",index=False)
PostPatchTweets_df.to_csv('PostPatchTweets.csv',na_rep="",index=False)
ffxiv_tweets_df.to_csv('FFXIVTweets.csv',na_rep="",index=False)